In [10]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
import yaml

pd.set_option("display.max_rows", None)

In [64]:
folder_to_harvest = Path("../data/douhet")

json_paths = folder_to_harvest.glob("**/*.json")

db = pd.DataFrame()

for data_path in json_paths:

    course_label =  data_path.stem.replace("_data", "")
    course, edition, ctx = course_label.split("_")

    associazione_path = Path(f"./data/{course}/{course}_{edition}/associazione.xlsx")
    associazione_df = pd.read_excel(associazione_path)
    
    with open(data_path, "r") as fin:
        
        data = json.loads(fin.read())
        root_key = list(data.keys())[0]
        
        sna_micro_stats_a = data[root_key]["sna"]["micro_stats_a"]
        sna_micro_stats_b = data[root_key]["sna"]["micro_stats_b"]
        sna_micro_sociogram = data[root_key]["sociogram"]["micro_stats"]
        
        sna_micro_stats_a_df = pd.DataFrame(sna_micro_stats_a).T.add_suffix("_a", axis=1)
        sna_macro_stats_b_df = pd.DataFrame(sna_micro_stats_b).T.add_suffix("_b", axis=1)
        sociogram_micro_stats_df = pd.DataFrame(sna_micro_sociogram).T
        
        course_df =  pd.concat([sna_micro_stats_a_df, sna_macro_stats_b_df, sociogram_micro_stats_df], axis=1)
        course_df = course_df.reset_index(names="id")
        course_df.insert(0, "corso", course)
        course_df.insert(1, "anno", edition)
        course_df.insert(3, "contesto", ctx)
        
        course_df = course_df.merge(associazione_df, left_on="id", right_on="lettera").drop("lettera", axis=1)
        columns_to_reorder =  list(course_df.columns)
        course_df = course_df.loc[:, [ *columns_to_reorder[:1], columns_to_reorder[-1], *columns_to_reorder[1:-1] ]]
    
    db = pd.concat([db, course_df])

db.soggetto = db.soggetto.str.lower()
db.loc[db.soggetto == "albanese michele", "soggetto"] = "albanese michelle"
db.loc[db.soggetto == "iovane daniele", "soggetto"] = "iovane daniele thomas"
db.loc[db.soggetto == "luongo beatrice", "soggetto"] = "luongo beatrice gaia azzurra"
db.loc[db.soggetto == "panicola giacomo christofer", "soggetto"] = "panicola giacomo christopher"
db.loc[db.soggetto == "suriano campagna", "soggetto"] = "suriano campagna mario"

In [65]:
long_db = db.copy()
long_db = long_db.rename(columns={"soggetto":  "id", "id": "lettera"})
c = long_db.columns
long_db = long_db.loc[:, [c[1], c[0], c[2], c[4], c[3], *c[5:] ]]
long_db = long_db.loc[long_db.corso == "vega", :].drop(columns=["corso"])
long_db.to_excel("abgrid_long.xlsx", index=False)
long_db.head()

,id,anno,contesto,lettera,lns_a,ic_a,pr_a,bt_a,cl_a,hu_a,...,mutual_preferences,mutual_rejections,balance,orientation,impact,affiliation_raw,attraction_raw,affiliation,attraction,status
0,abbamondi mario,i,stu,A,"N, S, W",0.0,0.004934,0.0,0.0,0.017801,...,0,0,-2,0,2,-2,0,92.25023,92.453141,underrated
1,albanese michelle,i,stu,B,"F, j, l",0.027027,0.042179,0.141479,0.335949,0.01975,...,0,1,0,0,2,0,1,95.977967,94.946886,underrated
2,bagatin giulio,i,stu,C,"M, R, V",0.054054,0.011243,0.014802,0.202878,0.025469,...,0,0,2,0,2,2,2,99.705705,97.44063,underrated
3,bonfanti francesca,i,stu,D,"j, R, g",0.378378,0.05173,0.091629,0.395484,0.042549,...,1,0,13,1,15,14,15,122.07213,129.859311,popular
4,ciliento giacomo pio,i,stu,E,"N, j, g",0.081081,0.038719,0.022472,0.343332,0.039374,...,0,0,3,3,3,6,3,107.16118,99.934375,-


In [66]:
wide_db = long_db.sort_values(by=["id", "anno", "contesto"])
wide_db = wide_db.pivot(index="id", columns=["contesto", "anno"])
wide_db.columns = ['_'.join(col).strip() for col in wide_db.columns.values]
wide_db.reset_index(names=["id"]).to_excel("abgrid_wide.xlsx", index=False, na_rep=np.nan)
wide_db

,lettera_lud_i,lettera_stu_i,lettera_lud_ii,lettera_stu_ii,lns_a_lud_i,lns_a_stu_i,lns_a_lud_ii,lns_a_stu_ii,ic_a_lud_i,ic_a_stu_i,...,affiliation_lud_ii,affiliation_stu_ii,attraction_lud_i,attraction_stu_i,attraction_lud_ii,attraction_stu_ii,status_lud_i,status_stu_i,status_lud_ii,status_stu_ii
id,,,,,,,,,,,,,,,,,,,,,
abbamondi mario,A,A,A,A,"N, P, e","N, S, W","J, Z, a","D, H, I",0.081081,0.0,...,105.222232,97.89923,104.019421,92.453141,102.650214,100.882583,controversial,underrated,-,controversial
albanese michelle,B,B,B,B,"F, Y, f","F, j, l","H, R, c","H, I, P",0.027027,0.027027,...,102.655372,95.868487,93.605466,94.946886,109.333362,95.194825,neglected,underrated,-,neglected
bagatin giulio,C,C,NaN,NaN,"P, M, V","M, R, V",NaN,NaN,0.081081,0.054054,...,NaN,NaN,97.076785,97.44063,NaN,NaN,-,underrated,NaN,NaN
bonfanti francesca,D,D,C,C,"a, d, l","j, R, g","I, P, W","P, V, W",0.027027,0.378378,...,94.954793,108.05295,93.605466,129.859311,95.967066,115.10198,neglected,popular,rejected,appreciated
ciliento giacomo pio,E,E,D,D,"C, M, G","N, j, g","I, L, X","I, U, b",0.135135,0.081081,...,115.489672,114.145182,107.49074,99.934375,112.674936,115.10198,-,-,appreciated,popular
cimini flaminia,F,F,NaN,NaN,"N, B, I","N, W, I",NaN,NaN,0.108108,0.027027,...,NaN,NaN,107.49074,94.946886,NaN,NaN,-,-,NaN,NaN
criscuolo lorenzo,G,G,E,E,"f, M, i","N, j, R","Z, a, D","K, N, Y",0.081081,0.027027,...,94.954793,97.89923,104.019421,94.946886,85.942344,92.350945,-,underrated,neglected,underrated
cristiano asia assunta,H,H,F,F,"C, J, Q","C, D, c","B, R, G","G, R, c",0.0,0.0,...,87.254213,77.591791,86.662829,92.453141,92.625492,89.507066,rejected,rejected,rejected,rejected
dalla nave eleonora,I,I,NaN,NaN,"N, F, W","W, M, V",NaN,NaN,0.054054,0.027027,...,NaN,NaN,100.548103,94.946886,NaN,NaN,underrated,underrated,NaN,NaN


In [74]:
wide_db[wide_db.isna().sum(axis=1)>0].reset_index(names="id").loc[:, ["id"]].to_excel("casi_da_attenzionare.xlsx", index=False)